<a href="https://colab.research.google.com/github/lrahbek/bachelor_scz/blob/main/analysis_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instrument overlap analysis using sentence transformer models

### Set up: package import

In [ ]:
pip install -U sentence-transformers

In [ ]:
# Import packages
import os
import numpy as np
import pandas as pd
import torch
import sentence_transformers
import pickle
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses, util, evaluation
from sentence_transformers.evaluation import TripletEvaluator, BinaryClassificationEvaluator

In [ ]:
# Set working directory
os.chdir("drive/MyDrive/Bachelor")

### Load in item files

In [ ]:
# Download items, codes and scale files
items_txt = open("items.txt", "r")
items_str = items_txt.read()
items = items_str.split("\n")
print(items)
items_txt.close()

codes_txt = open("code.txt", "r")
codes_str = codes_txt.read()
codes = codes_str.split("\n")
print(codes)
codes_txt.close()

scales_txt = open("scale.txt", "r")
scales_str = scales_txt.read()
scales = scales_str.split("\n")
print(scales)
scales_txt.close()

names_txt = open("name.txt", "r")
names_str = names_txt.read()
names = names_str.split("\n")
print(names)
names_txt.close()

['Unchanging Facial Expression - The subject\'s face appears wooden, mechanical, frozen. It doe not change expression, or changes less than normally expected, as the emotional content of discourse changes. Since phenothiazines may partially mimic this effect, the interviewer should be careful to not whether or not the subject is on medication, but should not try to "correct" the rating accordingly. Occasionally the subject’s expression is not as full as expected. Subject’s expressions are dulled overall, but not absent. Subject’s face has a flat “set” look, but flickers of affect arise occasionally. Subject’s face looks “wooden” and changes little, if at all throughout the interview.', 'Decreased Spontaneous Movements - The subject sits quietly throughout the interview and shows few or no spontaneous movements. He does not shift position, move his legs, move his hands, etc., or does so less than normally expected. Some decrease in spontaneous movements. Subject moves three or four time

### Load in data frame with manually coded overlap (0/1)


In [ ]:
# Download df with all symptom pairs and similairy column
bc_eval_df = pd.read_csv('bc_eval_df.csv')
code_item = { codes[i]: items[i] for i in range(len(codes))} #make dict of codes and items
bc_eval_df = bc_eval_df.replace(to_replace = code_item) #replace codes with text (items)

### Load in sentence transformer models

In [ ]:
# Download sentence transformer models
model1 = SentenceTransformer('all-mpnet-base-v1')
print("Max Sequence Length:", model1.max_seq_length)

model2 = SentenceTransformer('all-distilroberta-v1')
print("Max Sequence Length:", model2.max_seq_length)

model3 = SentenceTransformer("neuml/pubmedbert-base-embeddings")
print("Max Sequence Length:", model3.max_seq_length)


### Evaluate encodings on manually coded overlap

In [ ]:
evalbc = evaluation.BinaryClassificationEvaluator(sentences1 = (list(bc_eval_df["item1"])), sentences2 = (list(bc_eval_df["item2"])), labels = (list(bc_eval_df["bc_overlap"])), name = "BC_eval", batch_size = 16, show_progress_bar = True, write_csv = True)
print(model1.evaluate(evalbc, output_path= "/content/drive/MyDrive/Bachelor/"))
print(model2.evaluate(evalbc, output_path= "/content/drive/MyDrive/Bachelor/"))
print(model3.evaluate(evalbc, output_path= "/content/drive/MyDrive/Bachelor/"))

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

0.5805403707467288


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

0.5457112959128922


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

0.5447213012138723


### Encode embeddings and calculated cosine scores

In [ ]:
# Calculate sentence embeddings
embeddings1 = model1.encode(items,  convert_to_tensor=True)
for item, embedding in zip(items, embeddings1):
    print("Item:", item)
    print("Embedding:", embedding)
    print("")

embeddings2 = model2.encode(items,  convert_to_tensor=True)
for item, embedding in zip(items, embeddings2):
    print("Item:", item)
    print("Embedding:", embedding)
    print("")

embeddings3 = model3.encode(items,  convert_to_tensor=True)
for item, embedding in zip(items, embeddings3):
    print("Item:", item)
    print("Embedding:", embedding)
    print("")

In [ ]:
#Compute cosine similarities
cosine_scores1 = util.cos_sim(embeddings1, embeddings1)
cosine_scores2 = util.cos_sim(embeddings2, embeddings2)
cosine_scores3 = util.cos_sim(embeddings3, embeddings3)

### Export cosine scores for all pairs for each model

In [ ]:
# Export cosine scores to R
cos1_pd = pd.DataFrame(cosine_scores1)
cos1_pd.columns = codes
cos1_pd.index = codes
cos1_pd.to_csv("cossco1.csv", index=True)

cos2_pd = pd.DataFrame(cosine_scores2)
cos2_pd.columns = codes
cos2_pd.index = codes
cos2_pd.to_csv("cossco2.csv", index=True)

cos3_pd = pd.DataFrame(cosine_scores3)
cos3_pd.columns = codes
cos3_pd.index = codes
cos3_pd.to_csv("cossco3.csv", index=True)